In [2]:
hindi_file= "/data3/home/avneeshg/Self_study/MLDL/Learning_from_Machine_Learning/Projects/Machine_translator_Transformer/dataset/iitb-english-hindi/target_train.txt"
english_file= "/data3/home/avneeshg/Self_study/MLDL/Learning_from_Machine_Learning/Projects/Machine_translator_Transformer/dataset/iitb-english-hindi/source_train.txt"
 
START_TOKEN = "<START>"
PADDING_TOKEN = "<PAD"
END_TOKEN = "<END>"


In [3]:
english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', 
                      '*', '+', ',', '-', '.', '/', 
                     '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                     ':', '<', '=', '>', '?', '@', 
                     'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 
                     'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 
                     'Y', 'Z', '[', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e',
                     'f', 'g', 'h', 'i', 'j', 'k', 'l','m', 'n', 'o', 'p', 'q', 
                     'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', 
                     '~', PADDING_TOKEN, END_TOKEN]

hindi_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', 
                    '*', '+', ',', '-', '.', '/',
                    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                    ':', '<', '=', '>', '?', '@',
                    'ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ऑ', 'ओ', 'औ', 'क', 'ख', 'ग',
                    'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ',
                    'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', '़', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॅ',
                    'े', 'ै', 'ॉ', 'ो', 'ौ', '्', 'ॐ', '।', '॥', PADDING_TOKEN, END_TOKEN]

In [4]:
text = "नमनस्ते"
list(text)

['न', 'म', 'न', 'स', '्', 'त', 'े']

In [5]:
index_to_hindi = {k:v for k,v in enumerate(hindi_vocabulary)}
hindi_to_index = {v:k for k,v in enumerate(hindi_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [6]:
with open(english_file) as f:
    english_sentence = [line.strip() for line in f.readlines()]  # Removes "\n" from each line
    
with open(hindi_file) as f:
    hindi_sentence = [line.strip() for line in f.readlines()]  # Removes "\n" from each line

TOTAL_SENTENCES = len(english_sentence)


In [7]:
english_sentence[:10]

['Give your application an accessibility workout',
 'Accerciser Accessibility Explorer',
 'The default plugin layout for the bottom panel',
 'The default plugin layout for the top panel',
 'A list of plugins that are disabled by default',
 'Highlight duration',
 'The duration of the highlight box when selecting accessible nodes',
 'Highlight border color',
 'The color and opacity of the highlight border.',
 'Highlight fill color']

In [8]:
hindi_sentence[:10]

['अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें',
 'एक्सेर्साइसर पहुंचनीयता अन्वेषक',
 'निचले पटल के लिए डिफोल्ट प्लग-इन खाका',
 'ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका',
 'उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है',
 'अवधि को हाइलाइट रकें',
 'पहुंचनीय आसंधि (नोड) को चुनते समय हाइलाइट बक्से की अवधि',
 'सीमांत (बोर्डर) के रंग को हाइलाइट करें',
 'हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता।',
 'भराई के रंग को हाइलाइट करें']

In [9]:
max(len(x) for x in hindi_sentence), max(len(x) for x in english_sentence)

(8000, 11088)

In [10]:
PERCENTILE = 97
import numpy as np
print(f"{PERCENTILE} percentile of the length of the sentences in hindi is {np.percentile([len(x) for x in hindi_sentence], PERCENTILE)}")
print(f"{PERCENTILE} percentile of the length of the sentences in english is {np.percentile([len(x) for x in english_sentence], PERCENTILE)}")

97 percentile of the length of the sentences in hindi is 266.0
97 percentile of the length of the sentences in english is 271.0


In [11]:
max_sentence_length = 300

def is_valid_tokens(sentence, vocabulary):
    for token in list(sentence):
        if token not in vocabulary:
            return False
    return True

def is_valid_length(sentence, max_sentence_length):
    return len(list(sentence)) <= (max_sentence_length - 1) # need to reserve one space for <END> token


valid_sentence_indices = []
for index in range(len(hindi_sentence)):
    hindi_sen, english_sen = hindi_sentence[index], english_sentence[index]

    if is_valid_length(hindi_sen, max_sentence_length) \
        and is_valid_length(english_sen, max_sentence_length) \
        and is_valid_tokens(hindi_sen, hindi_vocabulary) \
        and is_valid_tokens(english_sen, english_vocabulary):
        valid_sentence_indices.append(index)
    



In [12]:
print(f"Number of sentences : {len(hindi_sentence)}")
print(f"Number of valid sentences : {len(valid_sentence_indices)}")

Number of sentences : 1659083
Number of valid sentences : 1337225


In [13]:
newhindi_sentence = [hindi_sentence[i] for i in valid_sentence_indices]
newenglish_sentence = [english_sentence[i] for i in valid_sentence_indices]


In [14]:
newhindi_sentence[:3], newenglish_sentence[:3]

(['अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें',
  'एक्सेर्साइसर पहुंचनीयता अन्वेषक',
  'निचले पटल के लिए डिफोल्ट प्लग-इन खाका'],
 ['Give your application an accessibility workout',
  'Accerciser Accessibility Explorer',
  'The default plugin layout for the bottom panel'])

In [15]:
import torch
from torch.utils.data import Dataset, DataLoader
class TextDataset(Dataset):
    def __init__(self, english_sentence, hindi_sentence):
        self.english_sentence = english_sentence
        self.hindi_sentence = hindi_sentence
    
    def __len__(self):
        return len(self.english_sentence)
    
    def __getitem__(self, index):
        return self.english_sentence[index], self.hindi_sentence[index]



In [16]:
dataset = TextDataset(newenglish_sentence, newhindi_sentence)
len(dataset)

1337225

In [17]:
dataset[1]

('Accerciser Accessibility Explorer', 'एक्सेर्साइसर पहुंचनीयता अन्वेषक')

# BATCHING

In [18]:
batch_size = 3
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
iterator = iter(train_loader)

In [19]:
for batch_num, batch in enumerate(iterator):
    print(batch)
    if batch_num > 3:
        break



[('Only one track at a time can be checked', 'It needs to increase the number of faculty particularly faculty with international experience.', 'We shared with one another our joys and our sorrows.'), ('सिर्फ एक ट्रैक को एक समय में जाँचा जा सकता है', 'इसे संकाय-सदस्यों की संख्या, विशेष रूप से अंतरराष्ट्रीय अनुभव वाले शिक्षकों की संख्या में वृद्धि करने की आवश्यकता है।', 'हमने सुख और दुःख साथ-साथ भोगे।')]
[('Please log onto - www. megamb. gov. in to avail the services', 'enlightened', 'rapidity'), ('इन सेवाओं का लाभ उठाने के लिए-... पर क्लिक करें।', 'बुद्ध', 'अप्रलंब')]
[('Justin McCarthy (American historian)', 'Do they not look in the dominion of the heavens and the earth and all things that Allah has created, and that it may be that the end of their lives is near. In what message after this will they then believe?', 'Number files according to a pattern'), ('जस्टिन मैकार्थी', 'क्या उन लोगों ने आसमान व ज़मीन की हुकूमत और ख़ुदा की पैदा की हुई चीज़ों में ग़ौर नहीं किया और न इस बात में कि या

In [20]:
def tokenize(sentence, language_to_index, start_token=True, end_token=True):
    sentence_word_indicies = [language_to_index[token] for token in list(sentence)]
    if start_token:
        sentence_word_indicies.insert(0, language_to_index[START_TOKEN])
    if end_token:
        sentence_word_indicies.append(language_to_index[END_TOKEN])

    for _ in range(len(sentence_word_indicies), max_sentence_length):
        sentence_word_indicies.append(language_to_index[PADDING_TOKEN])
    return torch.tensor(sentence_word_indicies)

In [21]:
batch

[('moon blindness', 'life', 'The Role of the Court'),
 ('निशांधता', 'जीवनी', 'अदालत की भूमिका')]

In [22]:
batch[0]

('moon blindness', 'life', 'The Role of the Court')

In [23]:
eng_tokenized, hindi_tokenized = [], []
for sentence_num in range(batch_size):
    eng_sentence, hn_sentence = batch[0][sentence_num], batch[1][sentence_num]
    eng_tokenized.append(tokenize(eng_sentence, english_to_index, start_token=False, end_token=False))
    hindi_tokenized.append(tokenize(hn_sentence, hindi_to_index, start_token=True, end_token=False))


In [24]:
eng_tokenized = torch.stack(eng_tokenized)
hindi_tokenized = torch.stack(hindi_tokenized)


In [25]:
eng_tokenized

tensor([[76, 78, 78, 77,  1, 65, 75, 72, 77, 67, 77, 68, 82, 82, 94, 94, 94, 94,
         94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94,
         94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94,
         94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94,
         94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94,
         94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94,
         94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94,
         94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94,
         94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94,
         94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94,
         94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94,
         94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94,
         94, 94, 94, 94, 94,

In [26]:
NEG_INFTY = -1e9

def create_masks(eng_batch, kn_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sentence_length, max_sentence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sentence_length, max_sentence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sentence_length, max_sentence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sentence_length, max_sentence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, kn_sentence_length = len(eng_batch[idx]), len(kn_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sentence_length)
      kn_chars_to_padding_mask = np.arange(kn_sentence_length + 1, max_sentence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, kn_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, kn_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, kn_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    print(f"encoder_self_attention_mask {encoder_self_attention_mask.size()}: {encoder_self_attention_mask[0, :10, :10]}")
    print(f"decoder_self_attention_mask {decoder_self_attention_mask.size()}: {decoder_self_attention_mask[0, :10, :10]}")
    print(f"decoder_cross_attention_mask {decoder_cross_attention_mask.size()}: {decoder_cross_attention_mask[0, :10, :10]}")
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [27]:
create_masks(batch[0], batch[1])

encoder_self_attention_mask torch.Size([3, 300, 300]): tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
decoder_self_attention_mask torch.Size([3, 300, 300]): tensor([[ 0.0000e+00, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09,
         -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09],
        [ 0.0000e+00,  0.0000e+00, -1.0000e+09, -1.0000e+09, -1.0000e+09,
         -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -1.0000e+09, -1.0000e+09,
         -1.0000e

(tensor([[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09],
          ...,
          [-1.0000e+09, -1.0000e+09, -1.0000e+09,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09],
          [-1.0000e+09, -1.0000e+09, -1.0000e+09,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09],
          [-1.0000e+09, -1.0000e+09, -1.0000e+09,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09]],
 
         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09],
          ...,
    

In [28]:
import torch.nn as nn
def get_device():
    return torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_sequence_length):
        super().__init__()
        self.d_model = d_model
        self.max_sequence_length = max_sequence_length

    def forward(self):
        even_i = torch.arange(0,self.d_model,2).float()    # even positions 
        denominator = torch.pow(10000,(even_i / self.d_model)) # 10000^(2i/d_model)
        position = torch.arange(self.max_sequence_length).reshape(self.max_sequence_length,1) # [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [10, 1]
        even_PE = torch.sin(position / denominator)
        odd_PE = torch.cos(position / denominator)
        PE = torch.zeros(self.max_sequence_length, self.d_model) # [10, 512]
        PE[:,0::2] = even_PE
        PE[:,1::2] = odd_PE
        return PE 

class SentenceEmbedding(nn.Module):
    "For a given sentence, create an embedding"
    def __init__(self, max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN):
        super().__init__()
        self.vocab_size = len(language_to_index)
        self.max_sequence_length = max_sequence_length
        self.embedding = nn.Embedding(self.vocab_size, d_model)
        self.language_to_index = language_to_index
        self.position_encoder = PositionalEncoding(d_model, max_sequence_length)
        self.dropout = nn.Dropout(p=0.1)
        self.START_TOKEN = START_TOKEN
        self.END_TOKEN = END_TOKEN
        self.PADDING_TOKEN = PADDING_TOKEN
    
    def batch_tokenize(self, batch, start_token=True, end_token=True):

        def tokenize(sentence, start_token=True, end_token=True):
            sentence_word_indicies = [self.language_to_index[token] for token in list(sentence)]
            if start_token:
                sentence_word_indicies.insert(0, self.language_to_index[self.START_TOKEN])
            if end_token:
                sentence_word_indicies.append(self.language_to_index[self.END_TOKEN])
            for _ in range(len(sentence_word_indicies), self.max_sequence_length):
                sentence_word_indicies.append(self.language_to_index[self.PADDING_TOKEN])
            return torch.tensor(sentence_word_indicies)

        tokenized = []
        for sentence_num in range(len(batch)):
           tokenized.append( tokenize(batch[sentence_num], start_token, end_token) )
        tokenized = torch.stack(tokenized)
        return tokenized.to(get_device())
    
    def forward(self, x, end_token=True): # sentence
        x = self.batch_tokenize(x ,end_token)
        x = self.embedding(x)
        pos = self.position_encoder().to(get_device())
        x = self.dropout(x + pos)
        return x

In [29]:
import torch
from model import Transformer

d_model = 512
batch_size = 30
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 200
hindi_vocab_size = len(hindi_vocabulary)

transformer = Transformer(d_model, 
                          ffn_hidden,
                          num_heads, 
                          drop_prob, 
                          num_layers, 
                          max_sequence_length,
                          hindi_vocab_size,
                          english_to_index,
                          hindi_to_index,
                          START_TOKEN, 
                          END_TOKEN, 
                          PADDING_TOKEN)

In [32]:
from torch import nn

criterian = nn.CrossEntropyLoss(ignore_index=hindi_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, hindi_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, hindi_batch)
        optim.zero_grad()
        
        hindi_predictions = transformer(eng_batch,
                                     hindi_batch,
                                     encoder_self_attention_mask.to(device), 
                                     decoder_self_attention_mask.to(device), 
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        
        labels = transformer.decoder.sentence_embedding.batch_tokenize(hindi_batch, start_token=False, end_token=True)
        loss = criterian(
            hindi_predictions.view(-1, hindi_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == hindi_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"Hindi Translation: {hindi_batch[0]}")
            kn_sentence_predicted = torch.argmax(hindi_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in kn_sentence_predicted:
              if idx == hindi_to_index[END_TOKEN]:
                break
              predicted_sentence += index_to_hindi[idx.item()]
            print(f"Hindi Prediction: {predicted_sentence}")


            transformer.eval()
            kn_sentence = ("",)
            eng_sentence = ("should we go to the mall?",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, kn_sentence)
                predictions = transformer(eng_sentence,
                                          kn_sentence,
                                          encoder_self_attention_mask.to(device), 
                                          decoder_self_attention_mask.to(device), 
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_hindi[next_token_index]
                kn_sentence = (kn_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                  break
            
            print(f"Evaluation translation (should we go to the mall?) : {kn_sentence}")
            print("-------------------------------------------")

Epoch 0
encoder_self_attention_mask torch.Size([3, 300, 300]): tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
decoder_self_attention_mask torch.Size([3, 300, 300]): tensor([[ 0.0000e+00, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09,
         -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09],
        [ 0.0000e+00,  0.0000e+00, -1.0000e+09, -1.0000e+09, -1.0000e+09,
         -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -1.0000e+09, -1.0000e+09,
         

TypeError: SentenceEmbedding.forward() takes from 2 to 3 positional arguments but 4 were given